In [1]:
import gzip
import networkx as nx
import cada
from networkx.generators.community import LFR_benchmark_graph
import math
from random import sample

## Amazon: snap Amazon 0601

In [2]:
edge_list = []
with gzip.open("amazon0601.txt.gz", mode="rt") as file:
    file_as_list = file.read().splitlines()
    for line in file_as_list:
        r = line.strip().split('\t')
        edge_list.append(tuple(r))
edge_list = edge_list[4:]

In [3]:
# G = nx.DiGraph(edge_list) # nodes: 403394, edges: 3387388

In [4]:
# undirected graph is used in the paper
G = nx.Graph(edge_list) # nodes: 403394, edges: 2443408

### cada: Louvain

In [5]:
Amazon_cdl = cada.cada(G).get_anomalies_threshold(4)

In [6]:
len(Amazon_cdl) # t5_294, 0.07%; t4_1374, 0.34%

1374

In [7]:
print(Amazon_cdl)

['1041', '50', '45', '49', '749', '669', '332368', '182140', '101782', '553', '1743', '383513', '386404', '379857', '376519', '295973', '225871', '319457', '269339', '226252', '177161', '156481', '270785', '304937', '148640', '203281', '308978', '157760', '308252', '105984', '193104', '254373', '144764', '156983', '196443', '151788', '67425', '161264', '48361', '239428', '111259', '109778', '256252', '35207', '33381', '52', '318781', '311503', '374632', '285570', '202919', '159200', '255560', '66174', '162047', '252873', '526', '1039', '942', '1297', '377943', '330571', '266379', '334566', '250643', '302761', '279037', '227676', '262548', '131946', '232298', '9237', '1963', '21697', '6740', '8934', '106646', '9239', '1345', '950', '719', '5019', '50499', '25714', '380036', '351764', '339441', '366124', '349711', '306413', '304304', '213990', '237019', '231773', '217224', '199137', '207078', '268864', '197772', '218627', '251811', '180244', '244856', '124819', '116706', '130701', '11444

### cada: Infomap

In [8]:
intG = nx.convert_node_labels_to_integers(G, label_attribute='origin')

In [9]:
Amazon_cdi = cada.cada(intG, algorithm='infomap').get_anomalies_threshold(5)

In [10]:
len(Amazon_cdi) # t5_1584, 0.39%

1584

In [11]:
Amazon_cdi_restored = []
intNameDic = nx.get_node_attributes(intG, 'origin')
for i in Amazon_cdi:
    Amazon_cdi_restored.append(intNameDic[i])

In [12]:
print(Amazon_cdi_restored)

['50', '1041', '669', '3135', '45', '530', '1297', '6740', '942', '533', '49', '10030', '2054', '1345', '52', '168175', '159113', '36836', '5018', '173023', '217881', '160269', '227679', '157712', '126620', '94451', '135208', '182140', '128950', '89938', '66641', '52877', '34391', '613', '50499', '8934', '1856', '950', '954', '1743', '15714', '383', '3992', '5781', '400802', '400259', '386626', '388333', '391658', '373354', '367910', '384692', '378609', '383513', '377125', '386404', '370388', '341137', '383530', '377118', '364613', '360015', '377519', '364815', '329679', '338326', '337490', '311103', '312807', '296726', '361447', '335716', '350262', '307701', '370474', '328977', '300511', '271640', '263026', '372284', '338078', '279231', '270410', '274072', '344924', '332602', '241276', '300051', '295973', '225871', '325433', '330068', '367873', '276102', '260480', '280292', '321857', '223576', '222773', '228965', '274188', '238116', '231686', '266386', '288983', '210182', '269339', '2

### Jaccard Similarity

In [13]:
#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = len(list(set(list1).union(list2)))
    return float(intersection) / union

jaccard(Amazon_cdl, Amazon_cdi_restored)

0.1478463329452852

# Synthetic network

In [28]:
n = 100 * 10**3 # [1, 500] * 10**3
tau1 = 3 # degree distribution
tau2 = 2 # community size distribution
mu = 0.4 # [0.1, 0.6]
k = 2 * n**1.15 / n # average_degree
k_max = int(n**(1/(tau1 - 1))) # max_degree
G_benchmank = LFR_benchmark_graph(n, tau1, tau2, mu, average_degree=k, max_degree=k_max, seed=10)

In [29]:
G_benchmank.number_of_nodes()

100000

In [30]:
G_benchmank.number_of_edges()

706780

## Replaced Anomaly

In [31]:
# generate the LFR benchmark graph
a = int(math.ceil(n/100/20) * ((2+21)*20/2))
G_ra = LFR_benchmark_graph(n+a, tau1, tau2, mu, average_degree=k, max_degree=k_max, seed=10)
# generate a list of anomaly nodes to replace 'a' nodes
anomaly_origin = []
for i in range(math.ceil(n/100)):
    anomaly_origin.append('Anomaly_'+str(i))
anomaly = [anomaly_origin[i: i+20] for i in range(0, len(anomaly_origin), 20)]

In [32]:
print(G_ra.number_of_nodes(), G_ra.number_of_edges(), len(anomaly))

111500 788022 50


In [33]:
# select all the nodes to be replaced and divide them into the [2, 21] nested lists
from itertools import islice

node_list = []
for node, d in G_ra.degree:
    if d < 2*k:
        node_list.append(node)
selected = sample(node_list, a)

selected = [selected[i: i+230] for i in range(0, len(selected), 230)]
length_to_split = list(range(2,22))
for chunk_230 in selected:
    Inputt = iter(chunk_230)
    selected[selected.index(chunk_230)] = [list(islice(Inputt, elem)) for elem in length_to_split]

In [34]:
for iteration in range(math.ceil(n/100/20)):
    for i in range(20):
        x_nodes = selected[iteration][i]
        for n in x_nodes:
            neighbors = list(G_ra.neighbors(n))
            for neighbor in neighbors:
                G_ra.add_edge(neighbor, anomaly[iteration][i])
            G_ra.remove_node(n)

In [35]:
print(G_ra.number_of_nodes(), G_ra.number_of_edges()) # to check whether anomaly graph is correctly generated

101000 786583


### cada: Louvain

In [36]:
replaced_cdl = cada.cada(G_ra).get_anomalies_threshold(5)

In [37]:
len(replaced_cdl)

7519

In [38]:
replaced_cdl

['Anomaly_76',
 'Anomaly_794',
 'Anomaly_874',
 'Anomaly_18',
 'Anomaly_192',
 'Anomaly_178',
 'Anomaly_637',
 'Anomaly_53',
 86642,
 83016,
 'Anomaly_172',
 'Anomaly_174',
 'Anomaly_851',
 9742,
 'Anomaly_336',
 'Anomaly_530',
 'Anomaly_657',
 'Anomaly_207',
 'Anomaly_499',
 'Anomaly_609',
 'Anomaly_234',
 80427,
 55565,
 10212,
 'Anomaly_508',
 'Anomaly_687',
 'Anomaly_66',
 'Anomaly_776',
 'Anomaly_929',
 'Anomaly_799',
 'Anomaly_175',
 'Anomaly_808',
 'Anomaly_901',
 'Anomaly_136',
 84157,
 62679,
 58996,
 46165,
 40267,
 17559,
 4159,
 430,
 334,
 'Anomaly_573',
 'Anomaly_787',
 'Anomaly_97',
 'Anomaly_714',
 'Anomaly_779',
 'Anomaly_890',
 'Anomaly_532',
 'Anomaly_453',
 'Anomaly_558',
 'Anomaly_512',
 'Anomaly_118',
 'Anomaly_936',
 'Anomaly_917',
 'Anomaly_337',
 'Anomaly_299',
 'Anomaly_374',
 'Anomaly_451',
 'Anomaly_647',
 'Anomaly_932',
 'Anomaly_158',
 'Anomaly_611',
 99587,
 94637,
 86153,
 80442,
 78776,
 68273,
 66054,
 64919,
 61884,
 60859,
 55022,
 52845,
 52339,
 46

In [49]:
recall = len(set(replaced_cdl).intersection(anomaly_origin)) / 1000
precision = len(set(replaced_cdl).intersection(anomaly_origin)) / len(replaced_cdl)
f1 = 2*recall*precision / (recall + precision)
print(recall, precision, f1)

0.776 0.1032052134592366 0.18218100716046484


### cada: Infomap

In [42]:
intG_ra = nx.convert_node_labels_to_integers(G_ra, label_attribute='origin')
replaced_cdi_origin = cada.cada(intG_ra, algorithm='infomap').get_anomalies_threshold(4)
replaced_cdi = []
intNameDic = nx.get_node_attributes(intG_ra, 'origin')
for i in replaced_cdi_origin:
    replaced_cdi.append(intNameDic[i])

In [43]:
len(replaced_cdi)

14922

In [45]:
replaced_cdi

['Anomaly_137',
 'Anomaly_197',
 'Anomaly_437',
 'Anomaly_537',
 'Anomaly_138',
 'Anomaly_816',
 'Anomaly_579',
 'Anomaly_255',
 'Anomaly_419',
 'Anomaly_318',
 'Anomaly_16',
 'Anomaly_573',
 'Anomaly_317',
 'Anomaly_399',
 'Anomaly_76',
 'Anomaly_936',
 'Anomaly_239',
 'Anomaly_234',
 'Anomaly_715',
 'Anomaly_279',
 'Anomaly_18',
 'Anomaly_277',
 'Anomaly_998',
 'Anomaly_219',
 'Anomaly_979',
 'Anomaly_778',
 'Anomaly_753',
 'Anomaly_115',
 'Anomaly_554',
 'Anomaly_738',
 'Anomaly_776',
 'Anomaly_999',
 'Anomaly_679',
 'Anomaly_954',
 'Anomaly_17',
 'Anomaly_198',
 'Anomaly_116',
 'Anomaly_839',
 'Anomaly_199',
 'Anomaly_858',
 'Anomaly_799',
 'Anomaly_794',
 'Anomaly_899',
 'Anomaly_757',
 'Anomaly_298',
 'Anomaly_975',
 'Anomaly_118',
 'Anomaly_439',
 'Anomaly_739',
 'Anomaly_458',
 'Anomaly_39',
 'Anomaly_99',
 'Anomaly_235',
 'Anomaly_874',
 'Anomaly_97',
 'Anomaly_617',
 'Anomaly_13',
 'Anomaly_958',
 'Anomaly_498',
 'Anomaly_779',
 'Anomaly_877',
 'Anomaly_175',
 'Anomaly_638',


In [46]:
len(set(replaced_cdi[:1000]).intersection(anomaly_origin))/1000

0.607

In [50]:
recall = len(set(replaced_cdi).intersection(anomaly_origin)) / 1000
precision = len(set(replaced_cdi).intersection(anomaly_origin)) / len(replaced_cdi)
f1 = 2*recall*precision / (recall + precision)
print(recall, precision, f1)

0.959 0.06426752446052808 0.12046225348574299
